In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import tensorflow as tf

from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
tf.config.experimental_run_functions_eagerly(True)

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


⚠️　Be sure to use **tensorflow.keras**, unless direct tensorflow is not available, and you can probably see any errors.<br>
Don't mix up **tf.keras** and **keras**.<br>

---

### How to make your own Loss Function.

Keras provides us a couple of ways to make Loss Fuction, but here main two ways will be explained.
1. `Function` (**Easy** way, but only if it's **simple one.**)
2. `Class` (You can make **anything complex**. Note that this is **not** so difficult.)

This notebook especially focuses on **Image Loss function**.  

ex) Reconstruction Loss, VGG Loss(Content Loss, Style Loss)...

#### 1. Function
`Function` type get **2** arguments.
- ***y_true* : Label of data.**  ex) Ground Truth image, numeric label, etc...
- ***y_pred* : Model prediction.**  This is the same type of *y_true*.

If your model is trained with image data, the shape of *y_true* and *y_pred* will be **(Batchsize, Imagesize,Imagesize, Channel)**.<br>
Of course you know **loss function gets batch data** because any model is trained with batch data.

##### 1-1. Simple MSE Loss
$$
Loss = \frac{1}{N} |y_{true}-y_{pred}|^2
$$


In [2]:
def simple_mse(y_true, y_pred):
    loss = tf.reduce_mean(tf.math.abs(y_true - y_pred)**2)
    # loss = K.mean(K.square(y_true - y_pred)) ← This is another option.
    return loss

Adding function in Loss function is possible.<br>
You can write simple MSE Loss like this as well.

In [3]:
def simple_mse_2(y_true, y_pred):
    def subtract(x,y):
        return tf.math.abs(x-y)
    def square(x):
        return x**2
    def reduce_mean(x):
        return tf.reduce_mean(x)
    loss = subtract(y_true, y_pred)
    loss = square(loss)
    loss = reduce_mean(loss)
    return loss

When you **pass arguments** to loss function before training, loss function have to be **wrapped** another function like this.

In [4]:
def bias_simple_loss(args):
    alpha = args[0]
    beta = args[1]
    def loss_function(y_true, y_pred):
        # Loss function that accepts (y_true, y_pred) must be placed inside.
        loss = tf.reduce_mean(tf.math.abs((y_true - y_pred)**2))
        return loss # This return value is referenced to the following overall return value.
    return loss_function

func_obj = bias_simple_loss([2,5]) 
# you set func_obj to fit
# ex) model.fit(x,y,loss=func_obj)

##### 1-2. VGG Loss
Defined as VGG Loss, there are many different types such as **Content Loss, Style Loss, etc**.<br>
The definition depends on the methodology which you are going to use.<br>
Therefore, with reference to the following example and papers, you can customize it.

**Below, a simple example of VGG Loss using one intermediate layer in VGG19.**

In [5]:
from tensorflow.keras.applications.vgg19 import VGG19

# image_shape must be the size of image used for training.
vgg19 = VGG19(include_top=False, weights='imagenet', input_shape=(32,32,3))

# Make VGG19 not train
vgg19.trainable = False
for l in vgg19.layers:
    l.trainable = False
    
# "block5_conv4" is used as intermediate layer to get features.
model = Model(inputs=vgg19.input, outputs=vgg19.get_layer('block5_conv4').output)
model.trainable = False

def vgg_loss(y_true, y_pred):
    return K.mean(K.square(model(y_true) - model(y_pred)))

Check what layers are in VGG19.

In [6]:
for i in range(len(vgg19.layers)):
    try:
        print(vgg19.layers[i].get_config()["activation"],model.layers[i].get_config()["name"])
    except:
        print(vgg19.layers[i].get_config()["name"])

input_1
relu block1_conv1
relu block1_conv2
block1_pool
relu block2_conv1
relu block2_conv2
block2_pool
relu block3_conv1
relu block3_conv2
relu block3_conv3
relu block3_conv4
block3_pool
relu block4_conv1
relu block4_conv2
relu block4_conv3
relu block4_conv4
block4_pool
relu block5_conv1
relu block5_conv2
relu block5_conv3
relu block5_conv4
block5_pool


**Example of Style Loss using Gram matrix.**<br>
This loss is just for an example but not useful and practical.

In [7]:
def style_loss(y_true,y_pred):
    
    shape = K.shape(y_true)
    B, C, H, W = shape[0], shape[1], shape[2], shape[3]
    size = H * W

    def gram_matrix(x):
        assert K.ndim(x) == 4, 'Input tensor should be a 4d (B, H, W, C) tensor'
        assert K.image_data_format() == 'channels_last', "Please use channels-last format"        

        x = K.permute_dimensions(x, (0, 3, 1, 2))
        features = K.reshape(x, K.stack([B, C, H*W])) # Permute channels and get resulting shape
        gram = K.batch_dot(features, features, axes=2) # Reshape x and do batch dot product
        # gram = gram / K.cast(C * H * W, x.dtype) # Normalize with channels, height and width
        return gram
    
    y_true_gram = gram_matrix(y_true)
    y_pred_gram = gram_matrix(y_pred)
    # return K.sum(K.square(y_true_gram - y_pred_gram)) / K.cast(size*B*C, dtype=tf.float32)
    return K.sum(K.abs(y_true_gram - y_pred_gram)) / K.cast(size*B*C, dtype=tf.float32)

# For debug
# style_loss(K.variable(X_train),K.variable(X_train[::-1]))

**Example of VGG Loss using multi intermediate layers in VGG19.**

*This is so complex loss fuuction that we need to use `Class` Loss function method instead of function type.*

In [8]:
from tensorflow.keras.applications import VGG19 
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model

# Feature Extractor
vgg19 = VGG19(include_top=False, weights='imagenet', input_shape=(32,32,3))
vgg19.trainable = False

# Intermediate layers in VGG19.
target_layers = [
    'block1_conv2', 
    'block2_conv2', 
    'block3_conv4', 
    'block4_conv4', 
    'block5_conv4'
]
# For weighting outputs of intermediate layers.
style_weights = np.array([1,1,1,1,1])

vgg19.outputs = [vgg19.get_layer(target_layers[i]).output for i in range(len(target_layers))]
style_model = Model(inputs=vgg19.input, outputs=vgg19.outputs)
style_model.trainable = False

In [9]:
def multiple_style_loss(y_true,y_pred): # VGG Style Loss (Multiple gram matrix)

    def style_loss(y_true_style, y_pred_style):

        shape = K.shape(y_true_style)
        B, H, W, C = shape[0], shape[1], shape[2], shape[3]
        size = H * W

        def gram_matrix(x):
            assert K.ndim(x) == 4, 'Input tensor should be a 4d (B, H, W, C) tensor'
            assert K.image_data_format() == 'channels_last', "Please use channels-last format"        

            x = K.permute_dimensions(x, (0, 3, 1, 2)) # (B, H, W, C) → (B, C, H, W)
            features = K.reshape(x, K.stack([B, C, H*W])) # Permute channels and get resulting shape
            gram = K.batch_dot(features, features, axes=2) # Reshape x and do batch dot product
            return gram/K.cast(size*C, dtype=tf.float32)

        y_true_gram = gram_matrix(y_true_style)
        y_pred_gram = gram_matrix(y_pred_style)

        return K.sum(K.square(y_true_gram - y_pred_gram)) / K.cast(size*C, dtype=tf.float32) 

    y_true_styles = style_model(y_true)
    y_pred_styles = style_model(y_pred)
    
    loss = K.zeros(shape=()) # Initialize the loss
    
    # sum up each style loss
    for y_true_style,y_pred_style,style_weight in zip(y_true_styles,y_pred_styles,style_weights):
        add_loss = K.variable(style_weight) * style_loss(y_true_style,y_pred_style)
        loss = K.update_add(loss, add_loss)

    return K.variable(loss)

# For debug
# multiple_style_loss(K.variable(x_train),K.variable(x_train[::-1]))

---
#### 2. Class
`Class` type consists of at least 2 functions.<br>
(This is common to classes written in pythons.)

1. **`__init__` (constructor)** : set instance parameter
2. **`call`** method : this will be called to **calculate loss** at every batch training.


**`call`** method get **2** arguments. Same as `Function` type.

- ***y_true* : Label of data.**  
- ***y_pred* : Model prediction.**  

If your model is trained with image data, the shape of *y_true* and *y_pred* will be **(Batchsize, Imagesize,Imagesize, Channel)**.<br>
Of course you know **loss function gets batch data** because any model is trained with batch data.

In [10]:
"""
class CustomLoss(tf.keras.losses.Loss):

    def __init__(self, args):
        super().__init__()

    def call(self, y_true, y_pred):
        # The process of calculating the loss
        return loss
"""

'\nclass CustomLoss(tf.keras.losses.Loss):\n\n    def __init__(self, args):\n        super().__init__()\n\n    def call(self, y_true, y_pred):\n        # The process of calculating the loss\n        return loss\n'

##### 
##### 
##### 2-1 Simple Loss function Class.
*Let's rewrite **style loss** in `Class` type.*

In [11]:
class VGG19Loss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()

        vgg19 = VGG19(include_top=False, weights='imagenet', input_shape=self.image_shape)
        vgg19.trainable = False
        for l in vgg19.layers:
            l.trainable = False
        model = Model(inputs=vgg19.input, outputs=vgg19.get_layer('block5_conv4').output)
        model.trainable = False

    def call(self, y_true, y_pred):
        return K.mean(K.square(model(y_true) - model(y_pred)))

**Example of VGG Loss using multi intermediate layers in VGG19.**

*Let's rewrite **multiple style loss** in `Class` type.*

In [12]:

class VGG19Loss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()

        vgg19 = VGG19(include_top=False, weights='imagenet', input_shape=(256,256,3))
        vgg19.trainable = False
        for l in vgg19.layers:
            l.trainable = False

        self.base_model = Model(inputs=vgg19.input, outputs=vgg19.get_layer("block4_conv2").output)
        self.base_model.trainable = False

        target_layers = ['block1_conv2', 'block2_conv2', 'block3_conv4', 'block4_conv4', 'block5_conv4']
        
        # Feature Extractor
        vgg19.outputs = [vgg19.get_layer(target_layers[i]).output for i in range(len(target_layers))]
        self.style_model = Model(inputs=vgg19.input, outputs=vgg19.outputs)
        self.style_model.trainable = False

        self.style_weights = np.array([1,1,1,1,1])      


    def call(self, y_true, y_pred):

        def style_loss(y_true_style, y_pred_style):

            shape = K.shape(y_true_style)
            B, H, W, C = shape[0], shape[1], shape[2], shape[3]
            size = H * W

            def gram_matrix(x):
                assert K.ndim(x) == 4, 'Input tensor should be a 4d (B, H, W, C) tensor'
                assert K.image_data_format() == 'channels_last', "Please use channels-last format"        

                x = K.permute_dimensions(x, (0, 3, 1, 2)) # (B, H, W, C) → (B, C, H, W)
                features = K.reshape(x, K.stack([B, C, H*W])) # Permute channels and get resulting shape
                gram = K.batch_dot(features, features, axes=2) # Reshape x and do batch dot product
                return gram/K.cast(size*C, dtype=tf.float32)

            y_true_gram = gram_matrix(y_true_style)
            y_pred_gram = gram_matrix(y_pred_style)
            
            return K.sum(K.square(y_true_gram - y_pred_gram)) / K.cast(size*C, dtype=tf.float32) 

        y_true_styles = self.style_model(y_true)
        y_pred_styles = self.style_model(y_pred)

        loss = K.zeros(shape=()) # Initialize the loss

        # sum up each style loss
        for y_true_style,y_pred_style,style_weight in zip(y_true_styles,y_pred_styles,self.style_weights):
            add_loss = K.variable(style_weight) * style_loss(y_true_style,y_pred_style)
            loss = K.update_add(loss, add_loss)

        return loss

##### 2-1 Multi Loss function Class.

It's already so easy to **combine multiple losses.**<br>
This is example to combine three losses below in one class.

- reconst_loss(): # MSE Loss
- content_loss(): # VGG Loss
- multiple_style_loss(): # VGG Style Loss (Multiple gram matrix)

In [13]:

class CombineLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()

        vgg19 = VGG19(include_top=False, weights='imagenet', input_shape=(256,256,3))
        vgg19.trainable = False
        for l in vgg19.layers:
            l.trainable = False

        ''' VGG Loss (Content Loss) '''        
        self.base_model = Model(inputs=vgg19.input, outputs=vgg19.get_layer("block4_conv2").output)
        self.base_model.trainable = False

        ''' Texture Loss '''
        target_layers = ['block1_conv2', 'block2_conv2', 'block3_conv4', 'block4_conv4', 'block5_conv4']
        
        vgg19.outputs = [vgg19.get_layer(target_layers[i]).output for i in range(len(target_layers))]
        self.style_model = Model(inputs=vgg19.input, outputs=vgg19.outputs)
        self.style_model.trainable = False

        self.style_weights = np.array([1,1,1,1,1])*(10**2)

        
    def call(self, y_true, y_pred):
        
        def reconst_loss(): # MSE Loss
            return K.mean(K.square(y_true - y_pred))

        def content_loss(): # VGG Loss
            vgg_true_output = self.base_model(y_true)
            vgg_pred_output = self.base_model(y_pred)
            return K.mean(K.square(vgg_true_output - vgg_pred_output))

        def multiple_style_loss(): # VGG Style Loss (Multiple gram matrix)
            def style_loss(y_true_style, y_pred_style):

                shape = K.shape(y_true_style)
                B, H, W, C = shape[0], shape[1], shape[2], shape[3]
                size = H * W

                def gram_matrix(x):
                    assert K.ndim(x) == 4, 'Input tensor should be a 4d (B, H, W, C) tensor'
                    assert K.image_data_format() == 'channels_last', "Please use channels-last format"        

                    x = K.permute_dimensions(x, (0, 3, 1, 2)) # (B, H, W, C) → (B, C, H, W)
                    features = K.reshape(x, K.stack([B, C, H*W])) # Permute channels and get resulting shape
                    gram = K.batch_dot(features, features, axes=2) # Reshape x and do batch dot product
                    return gram/K.cast(size*C, dtype=tf.float32)

                y_true_gram = gram_matrix(y_true_style)
                y_pred_gram = gram_matrix(y_pred_style)

                return K.sum(K.square(y_true_gram - y_pred_gram)) / K.cast(size*C, dtype=tf.float32) 

            y_true_styles = self.style_model(y_true)
            y_pred_styles = self.style_model(y_pred)

            loss = K.zeros(shape=()) # Initialize the loss
            
            # sum up each style loss
            for y_true_style,y_pred_style,style_weight in zip(y_true_styles,y_pred_styles,self.style_weights):
                add_loss = K.variable(style_weight) * style_loss(y_true_style,y_pred_style)
                loss = K.update_add(loss, add_loss)

            return loss
        
        # Loss function
        Loss = [
                content_loss(),
                reconst_loss(),
                multiple_style_loss()
        ]
        Loss = K.sum(Loss)

        return Loss

---
#### Try Loss Function.

Explaination has been over here 🎉<br>
Try any loss function above with AE(AutoEncoder) model.

In [14]:
def build_model():
    input_img = tf.keras.layers.Input(shape=(32, 32, 3))
    l1 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l1(10e-10))(input_img)
    l2 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l1(10e-10))(l1)
    l3 = tf.keras.layers.MaxPool2D(padding='same')(l2)
    l4 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l1(10e-10))(l3)
    l5 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l1(10e-10))(l4)
    l6 = tf.keras.layers.MaxPool2D(padding='same')(l5)
    l7 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l1(10e-10))(l6)
    l8 = tf.keras.layers.UpSampling2D()(l7)
    l9 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l1(10e-10))(l8)
    l10 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l1(10e-10))(l9)
    l11 = tf.keras.layers.add([l10, l5])
    l12 = tf.keras.layers.UpSampling2D()(l11)
    l13 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l1(10e-10))(l12)
    l14 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l1(10e-10))(l13)
    l15 = tf.keras.layers.add([l14, l2])
    
    encoder = tf.keras.models.Model(inputs=(input_img), outputs=l7)
    decoded_image = tf.keras.layers.Conv2D(3, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l1(10e-10))(l15)
    autoencoder = tf.keras.models.Model(inputs=(input_img), outputs=decoded_image)
    
    return autoencoder

In [15]:
from tensorflow.keras.datasets import cifar10
(x_train,y_train),(x_test,y_test)=cifar10.load_data()
x_train = x_train.astype('float32') / 255.

model = build_model()
################################################
 # loss = [Name of above losses or one you make]
################################################
model.compile(optimizer='adam', loss=CombineLoss())

# Auto Encoder image reconstruction training
model.fit(
    np.array(x_train)[:10],
    np.array(x_train)[:10],
    epochs=1,
    batch_size=2,
    shuffle=True)

/Users/macbookpro/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


5/5 [==============================] - 1s 174ms/step - loss: 11345.5479


---
#### References on Loss Function
- [Building Autoencoders in Keras](https://blog.keras.io/building-autoencoders-in-keras.html)
- [Shape preserving loss](https://github.com/ZengqiangYan/Shape-preservingLoss/blob/master/Loss.py)
- [Texture GAN](https://github.com/janesjanes/Pytorch-TextureGAN)
- [Style Loss](https://blog.shikoan.com/style-loss/)

#### References on Inplementation of Loss Function
- [tf.Keras Backend](https://www.tensorflow.org/api_docs/python/tf/keras/backend?hl=ja)
- [TensorFlow, KerasでVGG16などの学習済みモデルを利用](https://note.nkmk.me/python-tensorflow-keras-applications-pretrained-models/)
- [Extract Features, Visualize Filters and Feature Maps in VGG16 and VGG19 CNN Models](https://towardsdatascience.com/extract-features-visualize-filters-and-feature-maps-in-vgg16-and-vgg19-cnn-models-d2da6333edd0)
- [Loss Function Classの作成例](https://memo.sugyan.com/entry/2020/02/16/220750)
- [kerasで自作損失関数を実装したい](https://teratail.com/questions/185516)
- [Kerasで損失関数に複数の変数を渡す方法](https://blog.shikoan.com/keras-loss-function-multiple-arguments/)  
→ Custom Loss function usually receive only (y_true and y_pred).  
→ Q. When you get loss function to get other paramerters.  
→ A. Let's wrap the loss function with another function.

---
**Memo**

`<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=197785280.0>`

→ 'UnreadVariable' is handled dynamically when Tensor includes a calculation with just a number or so. <br>
→ The result is a Tensor form, but is not actually Tensor.<br>
→ Not necessary to care about here.

---
Thanks😎